# Scratchwork

This notebook is only used for trying out ideas.

In [1]:
import algebras as alg
import json
import os
import numpy as np
import itertools as it
# from pprint import pprint
import pprint as pp

# from itertools import combinations, permutations, product
import itertools as it

In [2]:
# Path to this repo
aa_path = os.path.join(os.getenv('PYPROJ'), 'abstract_algebra')

# Path to a directory containing Algebra definitions in JSON
alg_dir = os.path.join(aa_path, "Algebras")

## Groups for Testing

### D4 - Dihedral Group on 4 Vertices

In [3]:
d4_path = os.path.join(alg_dir, "d4_dihedral_group_on_4_vertices.json")
!cat {d4_path}

{"type": "Group",
 "name": "D_4",
 "description": "Dihedral group on four vertices",
 "element_names": ["e", "r", "r^2", "r^3", "f", "fr", "r^2f", "rf"],
 "alt_elem_names": ["()", "(0 1 2 3)", "(0 2)(1 3)", "(0 3 2 1)",
                    "(0 1)(2 3)", "(1 3)", "(0 3)(1 2)", "(0 2)"],
 "mult_table": [[0, 1, 2, 3, 4, 5, 6, 7],
                [1, 2, 3, 0, 7, 4, 5, 6],
                [2, 3, 0, 1, 6, 7, 4, 5],
                [3, 0, 1, 2, 5, 6, 7, 4],
                [4, 5, 6, 7, 0, 1, 2, 3],
                [5, 6, 7, 4, 3, 0, 1, 2],
                [6, 7, 4, 5, 2, 3, 0, 1],
                [7, 4, 5, 6, 1, 2, 3, 0]]
}

In [4]:
d4 = alg.Group(d4_path)
d4.about()


Group: D_4
Dihedral group on four vertices
Abelian? False
Elements:
   Index   Name   Inverse  Order
      0       e       e       1
      1       r     r^3       4
      2     r^2     r^2       2
      3     r^3       r       4
      4       f       f       2
      5      fr      fr       2
      6    r^2f    r^2f       2
      7      rf      rf       2
Cayley Table (showing indices):
[[0, 1, 2, 3, 4, 5, 6, 7],
 [1, 2, 3, 0, 7, 4, 5, 6],
 [2, 3, 0, 1, 6, 7, 4, 5],
 [3, 0, 1, 2, 5, 6, 7, 4],
 [4, 5, 6, 7, 0, 1, 2, 3],
 [5, 6, 7, 4, 3, 0, 1, 2],
 [6, 7, 4, 5, 2, 3, 0, 1],
 [7, 4, 5, 6, 1, 2, 3, 0]]


### V4 - Klein 4 Group

In [5]:
v4_path = os.path.join(alg_dir, "v4_klein_4_group.json")
!cat {v4_path}

{"type": "Group",
 "name": "V4",
 "description": "Klein-4 group",
 "element_names": ["e", "h", "v", "r"],
 "mult_table": [[0, 1, 2, 3],
                [1, 0, 3, 2],
                [2, 3, 0, 1],
                [3, 2, 1, 0]]
}


In [6]:
v4 = alg.Group(v4_path)
v4.about()


Group: V4
Klein-4 group
Abelian? True
Elements:
   Index   Name   Inverse  Order
      0       e       e       1
      1       h       h       2
      2       v       v       2
      3       r       r       2
Cayley Table (showing indices):
[[0, 1, 2, 3], [1, 0, 3, 2], [2, 3, 0, 1], [3, 2, 1, 0]]


In [7]:
z2 = alg.generate_cyclic_group(2)
z2

Group('Z2',
'Autogenerated cyclic group of order 2',
['e', 'a'],
[[0, 1], [1, 0]]) 

In [8]:
z2_x_z2 = z2 * z2
z2_x_z2

Group('Z2_x_Z2',
'Direct product of Z2 & Z2',
['e:e', 'e:a', 'a:e', 'a:a'],
[[0, 1, 2, 3], [1, 0, 3, 2], [2, 3, 0, 1], [3, 2, 1, 0]]) 

In [9]:
z4 = alg.generate_cyclic_group(4)
z4

Group('Z4',
'Autogenerated cyclic group of order 4',
['e', 'a', 'a^2', 'a^3'],
[[0, 1, 2, 3], [1, 2, 3, 0], [2, 3, 0, 1], [3, 0, 1, 2]]) 

### Powerset Group on 3 Items

TODO: Look for the paper by Carolyn Bean, "Group operations on the power set", Journal of Undergraduate Mathematics 8 #1 (March 1976), 13-17.

In [10]:
ps3 = alg.generate_powerset_group(3)
ps3.about()


Group: PS3
Autogenerated group on the powerset of 3 elements, with symmetric difference operator
Abelian? True
Elements:
   Index   Name   Inverse  Order
      0      {}      {}       1
      1     {0}     {0}       2
      2     {1}     {1}       2
      3     {2}     {2}       2
      4  {0, 1}  {0, 1}       2
      5  {0, 2}  {0, 2}       2
      6  {1, 2}  {1, 2}       2
      7 {0, 1, 2} {0, 1, 2}       2
Cayley Table (showing indices):
[[0, 1, 2, 3, 4, 5, 6, 7],
 [1, 0, 4, 5, 2, 3, 7, 6],
 [2, 4, 0, 6, 1, 7, 3, 5],
 [3, 5, 6, 0, 7, 1, 2, 4],
 [4, 2, 1, 7, 0, 6, 5, 3],
 [5, 3, 7, 1, 6, 0, 4, 2],
 [6, 7, 3, 2, 5, 4, 0, 1],
 [7, 6, 5, 4, 3, 2, 1, 0]]


## Finite Rings

### Set Operations

In [11]:
A = {0, 2, 4, 6, 8}
B = {1, 2, 3, 4, 5}

print(f"               Union: {A | B}")
print(f"        Intersection: {A & B}")
print(f"          Difference: {A - B}")
print(f"Symmetric Difference: {A ^ B}")

               Union: {0, 1, 2, 3, 4, 5, 6, 8}
        Intersection: {2, 4}
          Difference: {0, 8, 6}
Symmetric Difference: {0, 1, 3, 5, 6, 8}


"Consider any finite set. Then its power set with respect to symmetric difference (as 1st operation) and set intersection (as 2nd operation) is an example of a finite ring." -- [Quora](https://www.quora.com/What-are-examples-of-finite-rings#:~:text=Matrices%20over%20any%20ring%20make,a%20finite%20ring%20with%20elements.)

### Powerset Ring

In [12]:
psr = alg.generate_powerset_ring(3)

In [13]:
psr.add('{0}', '{1}')

'{0, 1}'

In [14]:
#psr.rmult_table

In [15]:
#psr.rmult_table_with_names()

In [16]:
psr.ring_mult('{0}', '{1}')

'{}'

In [17]:
%time psr.is_distributive()

CPU times: user 5 µs, sys: 4 µs, total: 9 µs
Wall time: 16.2 µs


True

In [18]:
psr.has_mult_identity

In [19]:
psr.mult_identity

'{0, 1, 2}'

In [20]:
psr.has_mult_identity

True

In [21]:
%time psr.is_distributive()

CPU times: user 5 µs, sys: 1e+03 ns, total: 6 µs
Wall time: 8.82 µs


True

## Summarize Proper Subgroups of a Group

In [22]:
psr3 = alg.generate_powerset_ring(3)

In [23]:
ps3_prop_subs = ps3.proper_subgroups()
print(f"\n{ps3.name} has {len(ps3_prop_subs)} proper subgroups.\n")


PS3 has 14 proper subgroups.



In [24]:
iso_sets_of_groups = alg.divide_groups_into_isomorphic_sets(ps3_prop_subs)

In [25]:
[[g.name for g in iso_set] for iso_set in iso_sets_of_groups]

[['PS3_subgroup_0',
  'PS3_subgroup_1',
  'PS3_subgroup_2',
  'PS3_subgroup_3',
  'PS3_subgroup_4',
  'PS3_subgroup_10',
  'PS3_subgroup_13'],
 ['PS3_subgroup_5',
  'PS3_subgroup_6',
  'PS3_subgroup_7',
  'PS3_subgroup_8',
  'PS3_subgroup_9',
  'PS3_subgroup_11',
  'PS3_subgroup_12']]

In [26]:
[iso_set[0].about() for iso_set in iso_sets_of_groups]


Group: PS3_subgroup_0
Subgroup of: Autogenerated group on the powerset of 3 elements, with symmetric difference operator
Abelian? True
Elements:
   Index   Name   Inverse  Order
      0      {}      {}       1
      1     {1}     {1}       2
Cayley Table (showing indices):
[[0, 1], [1, 0]]

Group: PS3_subgroup_5
Subgroup of: Autogenerated group on the powerset of 3 elements, with symmetric difference operator
Abelian? True
Elements:
   Index   Name   Inverse  Order
      0      {}      {}       1
      1     {0}     {0}       2
      2     {2}     {2}       2
      3  {0, 2}  {0, 2}       2
Cayley Table (showing indices):
[[0, 1, 2, 3], [1, 0, 3, 2], [2, 3, 0, 1], [3, 2, 1, 0]]


[None, None]

## Conjugation & Normal Subgroups

### V4

In [27]:
v4_subs = v4.proper_subgroups()
v4_subs

[Group('V4_subgroup_0',
 'Subgroup of: Klein-4 group',
 ['e', 'r'],
 [[0, 1], [1, 0]]) ,
 Group('V4_subgroup_1',
 'Subgroup of: Klein-4 group',
 ['e', 'v'],
 [[0, 1], [1, 0]]) ,
 Group('V4_subgroup_2',
 'Subgroup of: Klein-4 group',
 ['e', 'h'],
 [[0, 1], [1, 0]]) ]

In [28]:
v4.unique_proper_subgroups()

[Group('V4_subgroup_0',
 'Subgroup of: Klein-4 group',
 ['e', 'r'],
 [[0, 1], [1, 0]]) ]

In [29]:
v4.is_normal(v4_subs[0])

True

### D4

In [30]:
d4.about()


Group: D_4
Dihedral group on four vertices
Abelian? False
Elements:
   Index   Name   Inverse  Order
      0       e       e       1
      1       r     r^3       4
      2     r^2     r^2       2
      3     r^3       r       4
      4       f       f       2
      5      fr      fr       2
      6    r^2f    r^2f       2
      7      rf      rf       2
Cayley Table (showing indices):
[[0, 1, 2, 3, 4, 5, 6, 7],
 [1, 2, 3, 0, 7, 4, 5, 6],
 [2, 3, 0, 1, 6, 7, 4, 5],
 [3, 0, 1, 2, 5, 6, 7, 4],
 [4, 5, 6, 7, 0, 1, 2, 3],
 [5, 6, 7, 4, 3, 0, 1, 2],
 [6, 7, 4, 5, 2, 3, 0, 1],
 [7, 4, 5, 6, 1, 2, 3, 0]]


In [31]:
len(d4.proper_subgroups())

8

In [32]:
d4_subs = d4.unique_proper_subgroups()
d4_subs

[Group('D_4_subgroup_0',
 'Subgroup of: Dihedral group on four vertices',
 ['e', 'rf'],
 [[0, 1], [1, 0]]) ,
 Group('D_4_subgroup_1',
 'Subgroup of: Dihedral group on four vertices',
 ['e', 'r^2', 'fr', 'rf'],
 [[0, 1, 2, 3], [1, 0, 3, 2], [2, 3, 0, 1], [3, 2, 1, 0]]) ,
 Group('D_4_subgroup_3',
 'Subgroup of: Dihedral group on four vertices',
 ['e', 'r', 'r^2', 'r^3'],
 [[0, 1, 2, 3], [1, 2, 3, 0], [2, 3, 0, 1], [3, 0, 1, 2]]) ]

The result below matches the comment in the last paragraph of this section:
https://en.wikipedia.org/wiki/Dihedral_group#Properties

In [33]:
[d4.is_normal(sub) for sub in d4_subs]

[False, True, True]

### V4

In [34]:
grp = v4
grp_subs = grp.proper_subgroups()
[grp.is_normal(sub) for sub in grp_subs]

[True, True, True]

### PS3

In [35]:
grp = ps3
grp_unique_subs = grp.unique_proper_subgroups()
[grp.is_normal(sub) for sub in grp_unique_subs]

[True, True]

### WORK IN PROGRESS

In [36]:
subs = grp.proper_subgroups()
iso_subs = alg.divide_groups_into_isomorphic_sets(subs)
elem_name_lists = [[g.elements for g in iso_sub] for iso_sub in iso_subs]
elem_name_lists

[[['{}', '{1}'],
  ['{}', '{0, 1}'],
  ['{}', '{2}'],
  ['{}', '{0, 2}'],
  ['{}', '{0}'],
  ['{}', '{0, 1, 2}'],
  ['{}', '{1, 2}']],
 [['{}', '{0}', '{2}', '{0, 2}'],
  ['{}', '{1}', '{2}', '{1, 2}'],
  ['{}', '{2}', '{0, 1}', '{0, 1, 2}'],
  ['{}', '{1}', '{0, 2}', '{0, 1, 2}'],
  ['{}', '{0}', '{1}', '{0, 1}'],
  ['{}', '{0, 1}', '{0, 2}', '{1, 2}'],
  ['{}', '{0}', '{1, 2}', '{0, 1, 2}']]]

In [37]:
def about_proper_subgroups2(grp, unique=False, show_elements=True):
    if unique:
        subs = grp.unique_proper_subgroups()
    else:
        subs = grp.proper_subgroups()
    print(f"\nSubgroups of {grp.name}:")
    for sub in subs:
        print(f"\n  {sub.name}:")
        print(f"       order: {sub.order}")
        if show_elements:
            print(f"    elements: {sub.elements}")
        print(f"    abelian?: {sub.is_abelian()}")
        print(f"     normal?: {grp.is_normal(sub)}")
    return None

In [38]:
def about_proper_subgroups3(grp, unique=False, show_elements=True):
    subs = grp.proper_subgroups()
    if unique:
        iso_subs = alg.divide_groups_into_isomorphic_sets(subs)
        elem_name_lists = [g.element_names for g in iso_subs[0]]
        elem_name_lists[1:]
    print(f"\nSubgroups of {grp.name}:")
    for sub in subs:
        print(f"\n  {sub.name}:")
        print(f"       order: {sub.order}")
        if show_elements:
            print(f"    elements: {sub.elements}")
        print(f"    abelian?: {sub.is_abelian()}")
        print(f"     normal?: {grp.is_normal(sub)}")
    return None

In [39]:
about_proper_subgroups2(ps3, unique=True)


Subgroups of PS3:

  PS3_subgroup_0:
       order: 2
    elements: ['{}', '{1}']
    abelian?: True
     normal?: True

  PS3_subgroup_5:
       order: 4
    elements: ['{}', '{0}', '{2}', '{0, 2}']
    abelian?: True
     normal?: True


In [40]:
ps3 = alg.generate_powerset_group(3)
ps3.about()


Group: PS3
Autogenerated group on the powerset of 3 elements, with symmetric difference operator
Abelian? True
Elements:
   Index   Name   Inverse  Order
      0      {}      {}       1
      1     {0}     {0}       2
      2     {1}     {1}       2
      3     {2}     {2}       2
      4  {0, 1}  {0, 1}       2
      5  {0, 2}  {0, 2}       2
      6  {1, 2}  {1, 2}       2
      7 {0, 1, 2} {0, 1, 2}       2
Cayley Table (showing indices):
[[0, 1, 2, 3, 4, 5, 6, 7],
 [1, 0, 4, 5, 2, 3, 7, 6],
 [2, 4, 0, 6, 1, 7, 3, 5],
 [3, 5, 6, 0, 7, 1, 2, 4],
 [4, 2, 1, 7, 0, 6, 5, 3],
 [5, 3, 7, 1, 6, 0, 4, 2],
 [6, 7, 3, 2, 5, 4, 0, 1],
 [7, 6, 5, 4, 3, 2, 1, 0]]


In [41]:
foo = ['e', 'a', 'b', 'c', 'd', 'f', 'g', 'h']

rev_mapping = dict(zip(foo, ps3.elements))
mapping = dict(zip(ps3.elements, foo))

In [42]:
ps3_psubs = ps3.proper_subgroups()
ps3_psubs

[Group('PS3_subgroup_0',
 'Subgroup of: Autogenerated group on the powerset of 3 elements, with symmetric difference operator',
 ['{}', '{1}'],
 [[0, 1], [1, 0]]) ,
 Group('PS3_subgroup_1',
 'Subgroup of: Autogenerated group on the powerset of 3 elements, with symmetric difference operator',
 ['{}', '{0, 1}'],
 [[0, 1], [1, 0]]) ,
 Group('PS3_subgroup_2',
 'Subgroup of: Autogenerated group on the powerset of 3 elements, with symmetric difference operator',
 ['{}', '{2}'],
 [[0, 1], [1, 0]]) ,
 Group('PS3_subgroup_3',
 'Subgroup of: Autogenerated group on the powerset of 3 elements, with symmetric difference operator',
 ['{}', '{0, 2}'],
 [[0, 1], [1, 0]]) ,
 Group('PS3_subgroup_4',
 'Subgroup of: Autogenerated group on the powerset of 3 elements, with symmetric difference operator',
 ['{}', '{0}'],
 [[0, 1], [1, 0]]) ,
 Group('PS3_subgroup_5',
 'Subgroup of: Autogenerated group on the powerset of 3 elements, with symmetric difference operator',
 ['{}', '{0}', '{2}', '{0, 2}'],
 [[0, 1,

In [43]:
ps3_psubs_x = [psub.set_elements(mapping) for psub in ps3_psubs]

In [44]:
foo = sorted(ps3_psubs_x, key=lambda x: x.elements)

In [45]:
[z.elements for z in foo]

[['e', 'a'],
 ['e', 'a', 'b', 'd'],
 ['e', 'a', 'c', 'f'],
 ['e', 'a', 'g', 'h'],
 ['e', 'b'],
 ['e', 'b', 'c', 'g'],
 ['e', 'b', 'f', 'h'],
 ['e', 'c'],
 ['e', 'c', 'd', 'h'],
 ['e', 'd'],
 ['e', 'd', 'f', 'g'],
 ['e', 'f'],
 ['e', 'g'],
 ['e', 'h']]

In [46]:
#rmap = ps3.set_elements(['e', 'a', 'b', 'c', 'd', 'f', 'g', 'h'])
rmap = ps3.set_elements(mapping)
ps3.about()


Group: PS3
Autogenerated group on the powerset of 3 elements, with symmetric difference operator
Abelian? True
Elements:
   Index   Name   Inverse  Order
      0       e       e       1
      1       a       a       2
      2       b       b       2
      3       c       c       2
      4       d       d       2
      5       f       f       2
      6       g       g       2
      7       h       h       2
Cayley Table (showing indices):
[[0, 1, 2, 3, 4, 5, 6, 7],
 [1, 0, 4, 5, 2, 3, 7, 6],
 [2, 4, 0, 6, 1, 7, 3, 5],
 [3, 5, 6, 0, 7, 1, 2, 4],
 [4, 2, 1, 7, 0, 6, 5, 3],
 [5, 3, 7, 1, 6, 0, 4, 2],
 [6, 7, 3, 2, 5, 4, 0, 1],
 [7, 6, 5, 4, 3, 2, 1, 0]]


In [47]:
mapping

{'{}': 'e',
 '{0}': 'a',
 '{1}': 'b',
 '{2}': 'c',
 '{0, 1}': 'd',
 '{0, 2}': 'f',
 '{1, 2}': 'g',
 '{0, 1, 2}': 'h'}

In [49]:
#sub2 = ps3_subs[2]
#sub2
#ps3_subs_x = [sub.set_elements(mapping) for sub in ps3_subs]

## Cayley Table Class

In [178]:
import numpy as np

class CayleyTable:

    def __init__(self, arr):
        if isinstance(arr, list):
            if all(isinstance(row, list) for row in arr):
                if all(len(row) == len(arr) for row in arr):
                    self.__order = len(arr)
                    self.__table = np.array(arr)
                else:
                    raise Exception("Input must represent a square tabl")
            else:
                raise Exception("All table rows must be lists")
        else:
            raise Exception(f"Single argument type, {type(arr)}, not supported")

    def __repr(self):
        print(f"CayleyTable(\n{self.__table}\n)")
    
    def __getitem__(self, tup):
        row, col = tup
        return self.__table[row][col]

    @property
    def order(self):
        return self.__order
    
    @property
    def table(self):
        return self.__table

    def is_associative(self):
        indices = range(len(self.__table))
        result = True
        for a in indices:
            for b in indices:
                for c in indices:
                    ab = self.__table[a][b]
                    bc = self.__table[b][c]
                    if not (self.__table[ab][c] == self.__table[a][bc]):
                        result = False
                        break
        return result

    def is_commutative(self):
        indices = range(len(self.__table))
        result = True
        for a in indices:
            for b in indices:
                if self.__table[a][b] != self.__table[b][a]:
                    result = False
                    break
        return result

    def left_identity(self):
        indices = range(len(self.__table))
        identity = None
        for x in indices:
            if all(self.__table[x][y] == y for y in indices):
                identity = x
                break
        return identity

    def right_identity(self):
        indices = range(len(self.__table))
        identity = None
        for x in indices:
            if all(self.__table[y][x] == y for y in indices):
                identity = x
                break
        return identity

    def identity(self):
        left_id = self.left_identity()
        right_id = self.right_identity()
        if (left_id is not None) and (right_id is not None):
            return left_id
        else:
            return None

    # def has_inverses(table):
    #     return False

    def inverse_lookup_dict(self, identity):
        elements = range(len(self.__table))
        row_indices, col_indices = np.where(self.__table == identity)
        return {elements[elem_index]: elements[elem_inv_index]
                for (elem_index, elem_inv_index)
                in zip(row_indices, col_indices)}

In [179]:
# not assoc; is comm; no identities -- the RPS magma table, above
arr1 = [[0, 1, 0], [1, 1, 2], [0, 2, 2]]

# is assoc; not comm; has identity (0) --- the S3 group table
arr2 = [[0, 1, 2, 3, 4, 5], [1, 2, 0, 5, 3, 4], [2, 0, 1, 4, 5, 3],
        [3, 4, 5, 0, 1, 2], [4, 5, 3, 2, 0, 1], [5, 3, 4, 1, 2, 0]]

# is assoc; is comm; has identity (0) --- the Z4 group table
arr3 = [[0, 1, 2, 3], [1, 2, 3, 0], [2, 3, 0, 1], [3, 0, 1, 2]]

# is assoc; is comm; has identity (0) --- powerset(3) group table
arr4 = [[0, 1, 2, 3, 4, 5, 6, 7], [1, 0, 4, 5, 2, 3, 7, 6], [2, 4, 0, 6, 1, 7, 3, 5],
        [3, 5, 6, 0, 7, 1, 2, 4], [4, 2, 1, 7, 0, 6, 5, 3], [5, 3, 7, 1, 6, 0, 4, 2],
        [6, 7, 3, 2, 5, 4, 0, 1], [7, 6, 5, 4, 3, 2, 1, 0]]

arr5 = [[0, 3, 0, 3, 0, 3], [1, 4, 1, 4, 1, 4], [2, 5, 2, 5, 2, 5],
        [3, 0, 3, 0, 3, 0], [4, 1, 4, 1, 4, 1], [5, 2, 5, 2, 5, 2]]

# is assoc; is not comm; no left id; has right id --- Smarandache Groupoid
test_arrays = [arr1, arr2, arr3, arr4, arr5]
test_tables = [CayleyTable(arr) for arr in test_arrays]

In [180]:
print("   Table  Order  Associative?  Commutative?  Left Id?  Right Id?  Identity?")
print('-' * 75)
for tbl in test_tables:
    i = test_tables.index(tbl) + 1
    order = str(tbl.order)
    is_assoc = str(tbl.is_associative())
    is_comm = str(tbl.is_commutative())
    lft_id = str(tbl.left_identity())
    rgt_id = str(tbl.right_identity())
    ident = str(tbl.identity())
    print(f"{i :>{6}} {order :>{6}} {is_assoc :>{11}} {is_comm :>{12}} {lft_id :>{12}} {rgt_id :>{9}} {ident :>{10}}")

   Table  Order  Associative?  Commutative?  Left Id?  Right Id?  Identity?
---------------------------------------------------------------------------
     1      3       False         True         None      None       None
     2      6        True        False            0         0          0
     3      4        True         True            0         0          0
     4      8        True         True            0         0          0
     5      6        True        False         None         0       None
